# searching a STAC catalog for different kinds of data

This notebook aims to demonstrate how to search different kinds of data in a STAC catalog, using the same search criteria.

For this, we need:
- `shapely` to define the search geometry
- `pystac-client` to query the STAC API
- `stac-geoparquet` to store the queried items as `geoparquet` files
- `geopandas` to work around an issue with `stac-geoparquet`
- `pyarrow` to read the `geoparquet` files
- `lonboard` to visualize the geometries
- `stac-insitu` to filter trajectories

In [ ]:
import json
import pathlib

import geopandas as gpd
import lonboard
import numpy as np
import pyarrow.parquet as pq
import pystac
import pystac_client
import shapely
import stac_geoparquet
from matplotlib import colormaps
from stac_insitu.filter import filter_trajectories

cache_root = pathlib.Path.home() / "work/data/stac/cache"

some functions for easy I/O

In [ ]:
def dump_to_ndjson(collection, path):
    with open(path, "w") as f:
        for item in collection:
            json.dump(item.to_dict(), f, separators=(",", ":"))
            f.write("\n")


def read_ndjson(path):
    """read geometries and IDs from a ndjson file"""
    with open(path) as f:
        items = [pystac.Item.from_dict(json.loads(line)) for line in f]

    geometries = [shapely.from_geojson(json.dumps(item.geometry)) for item in items]
    collection_ids = [item.collection_id for item in items]
    item_ids = [item.id for item in items]
    return gpd.GeoDataFrame(
        {"collection": collection_ids, "id": item_ids},
        geometry=geometries,
        crs="epsg:4326",
    )


def write_to_geoparquet(collection, cache_root, path):
    cache_path = cache_root.joinpath(path.name).with_suffix(".jsonl")
    dump_to_ndjson(collection, cache_path)
    stac_geoparquet.arrow.parse_stac_ndjson_to_parquet(cache_path, path)

## connect to the STAC API

In [ ]:
client = pystac_client.Client.open("http://localhost:9588")
client

## definition of the search

The search consists of a bbox (represented by a polygon) and a timespan (represented by start / stop times).

In [ ]:
bbox = shapely.box(-8, 46, 1, 51)
datetime = ["2022-05-10T00:00:00", "2022-05-12T00:00:00"]

## search for satellite data

This is pretty standard: provide the search criteria and the collections to search, and fetch the result.

In [ ]:
collections = ["AVHRR_SST_METOP_B-OSISAF-L2P-v1.0"]
image_items = client.search(
    collections=collections,
    intersects=bbox,
    datetime=datetime,
).item_collection()
image_items

to be able to reuse the search results without having the STAC API available, we'll store them to [`stac-geoparquet`](https://github.com/stac-utils/stac-geoparquet) (a way of serializing STAC items to `geoparquet`)

In [ ]:
sst_path = cache_root.joinpath("avhrr-sst-metop_b.parquet")
write_to_geoparquet(image_items, cache_root, sst_path)

reading them back allows us to visualize and inspect the result:

In [ ]:
table = pq.read_table(sst_path)

In [ ]:
lonboard.viz(table)

as we can see, there's an issue with images that cross the dateline: the bounding box (as given by the dataset's attributes) spans the entire earth. To fix that, we'd have to change the way the geometry was inferred when creating the catalog.

## search for in-situ data

Since the STAC catalog contains in-situ collections, we can query that in the same way (the only difference is the collection names). Additionally, we can use `stac-insitu`'s filter function to make sure trajectories actually intersect during the time we want it to intersect.

In [ ]:
catalog_id = "INSITU_GLO_PHYBGCWAV_DISCRETE_MYNRT_013_030"
short_names = ["BO", "CT", "DB", "FB", "SM", "DC", "MO", "RF", "TS", "TG", "GL"]
category_names = {
    "bo": "bottles",
    "ct": "conductivity, temperature, and depth sensors (CTD)",
    "db": "drifting buoys",
    "dc": "drifters",
    "fb": "ferrybox",
    "gl": "gliders",
    "hf": "high frequency radars",
    "ml": "mini loggers",
    "mo": "moorings",
    "pf": "profilers",
    "rf": "river flows",
    "sd": "saildrones",
    "sm": "sea mammals",
    "tg": "tide gauges",
    "ts": "thermosalinometer",
    "tx": "thermistor chains",
    "xb": "expendable bathythermographs (XBT)",
}
long_names = [category_names[n.lower()] for n in short_names]

collections = [f"{catalog_id}-{col}" for col in short_names]
collections

In [ ]:
all_insitu_items = client.search(
    collections=collections,
    intersects=bbox,
    datetime=datetime,
).item_collection()
insitu_items = filter_trajectories(all_insitu_items, bbox, datetime)
len(insitu_items)

Since `stac-geoparquet` does not support writing multiple collections into the same file at the moment, and also appears to choke on a geometry collection (i.e. items with different kinds of geometries), we have to write to `ndjson` instead: 

In [ ]:
insitu_path = cache_root.joinpath(
    "insitu_global_phybgcwav_discrete_mynrt_013_030.jsonl"
)
# `stac-geoparquet` currently chokes on linestring objects / multiple collections, so we have to use the `ndjson` file
dump_to_ndjson(insitu_items, insitu_path)

we can then read it back into memory and prepare for visualization:

In [ ]:
translation_table = dict(zip(collections, long_names))
table = read_ndjson(insitu_path).assign(
    long_names=lambda df: df["collection"].map(lambda it: translation_table[it])
)

In [ ]:
categories = table["collection"].unique()
cmap = dict(zip(categories, colormaps["Pastel1"].colors))
colors = np.stack(
    table["collection"]
    .map(lambda it: np.asarray(cmap[it], dtype=float) * 255)
    .to_list()
).astype("uint8")

geom_types = shapely.get_type_id(table.geometry).to_numpy()

scatterplot_colors = colors[geom_types == shapely.GeometryType.POINT, :]
path_colors = colors[geom_types == shapely.GeometryType.LINESTRING, :]

kwargs = {
    "scatterplot_kwargs": {"get_fill_color": scatterplot_colors},
    "path_kwargs": {"get_color": path_colors},
}

In [ ]:
lonboard.viz(table, **kwargs)